In [3]:
pip install flask_sqlalchemy

  Obtaining dependency information for flask_sqlalchemy from https://files.pythonhosted.org/packages/1d/6a/89963a5c6ecf166e8be29e0d1bf6806051ee8fe6c82e232842e3aeac9204/flask_sqlalchemy-3.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for flask>=2.2.5 from https://files.pythonhosted.org/packages/61/80/ffe1da13ad9300f87c93af113edd0638c75138c42a0994becfacac078c06/flask-3.0.3-py3-none-any.whl.metadata
  Obtaining dependency information for sqlalchemy>=2.0.16 from https://files.pythonhosted.org/packages/cd/ae/062f6ebd474aef81a199a16d2b1fb521d5fb0bc38a470181b0bcbfe3eb11/SQLAlchemy-2.0.30-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for Werkzeug>=3.0.0 from https://files.pythonhosted.org/packages/9d/6e/e792999e816d19d7fcbfa94c730936750036d65656a76a5a688b57a656c4/werkzeug-3.0.3-py3-none-any.whl.metadata
  Obtaining dependency information for itsdangerous>=2.1.2 from https://files.pythonhosted.org/packages/04/96/92447566d16df59b2a776c0fb82dbc4

In [5]:
from flask import Flask, render_template, request, redirect, url_for, session, flash
from flask_sqlalchemy import SQLAlchemy
from werkzeug.security import generate_password_hash, check_password_hash

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'  # Secret key for session management
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///quizapp.db'  # SQLite database URI
db = SQLAlchemy(app)

# Database Models

class User(db.Model):
    """User model for storing user details"""
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(150), unique=True, nullable=False)
    password = db.Column(db.String(150), nullable=False)
    is_teacher = db.Column(db.Boolean, default=False)  # Boolean flag to differentiate teachers from students

class Class(db.Model):
    """Class model for storing class details"""
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(150), nullable=False)
    teacher_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    students = db.relationship('User', secondary='class_student', backref='classes')  # Many-to-Many relationship with User

class ClassStudent(db.Model):
    """Association table for many-to-many relationship between Class and User"""
    id = db.Column(db.Integer, primary_key=True)
    class_id = db.Column(db.Integer, db.ForeignKey('class.id'), nullable=False)
    student_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)

class Quiz(db.Model):
    """Quiz model for storing quiz details"""
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(150), nullable=False)
    creator_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    questions = db.relationship('Question', backref='quiz')  # One-to-Many relationship with Question

class Question(db.Model):
    """Question model for storing quiz questions"""
    id = db.Column(db.Integer, primary_key=True)
    text = db.Column(db.String(500), nullable=False)
    answer = db.Column(db.String(150), nullable=False)
    quiz_id = db.Column(db.Integer, db.ForeignKey('quiz.id'), nullable=False)

# Routes

@app.route('/')
def home():
    """Home route displaying the main page"""
    return render_template('home.html')

@app.route('/register', methods=['GET', 'POST'])
def register():
    """Route for user registration"""
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        is_teacher = 'is_teacher' in request.form  # Check if the user is a teacher
        hashed_password = generate_password_hash(password, method='sha256')  # Hash the password
        new_user = User(username=username, password=hashed_password, is_teacher=is_teacher)
        db.session.add(new_user)
        db.session.commit()  # Save the new user to the database
        flash('Registration successful!', 'success')
        return redirect(url_for('login'))
    return render_template('register.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    """Route for user login"""
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        user = User.query.filter_by(username=username).first()
        if user and check_password_hash(user.password, password):  # Verify the password
            session['user_id'] = user.id
            session['is_teacher'] = user.is_teacher
            flash('Login successful!', 'success')
            return redirect(url_for('dashboard'))
        flash('Login failed. Check your credentials and try again.', 'danger')
    return render_template('login.html')

@app.route('/logout')
def logout():
    """Route for user logout"""
    session.pop('user_id', None)
 
